Chainlit used to create chat interface
Link: https://docs.chainlit.io/integrations/langchain

Useful Resources:
* Memory: https://www.youtube.com/watch?v=X550Zbz_ROE


In [ ]:
#!pip install huggingface_hub
#!pip install transformers
#!pip install langchain
#!pip install chainlit
#!pip install openai

In [1]:
#import openai
#import chainlit as cl
import os
from langchain import HuggingFaceHub, PromptTemplate, LLMChain
from langchain.chains import ConversationChain
from dotenv import load_dotenv

load_dotenv('.env')

True

In [2]:
# model_id = "gpt2-medium" # 355M parameter
# model_id = "google/gemma-2b-it"
# model_id = "google/gemma-7b"
model_id = "google/flan-ul2"
temperature = 0.1

conv_model = HuggingFaceHub(huggingfacehub_api_token=os.environ['HUGGINGFACEHUB_API_TOKEN'],
                            repo_id=model_id,
                            model_kwargs={"temperature": temperature, "max_new_tokens":200}) # Temperature is randomness of model

c:\Anaconda3\lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'InferenceApi' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


In [6]:
from langchain.chains.conversation.memory import ConversationBufferMemory, ConversationSummaryMemory, ConversationSummaryBufferMemory, ConversationKGMemory, ConversationEntityMemory 
memory = ConversationBufferMemory()
conv_chain = ConversationChain(llm=conv_model,
                               verbose=True,
                               memory=memory)

In [7]:
conv_chain.predict(input="Hi, I'm Julian. My TV is broken but is under warranty")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, I'm Julian. My TV is broken but is under warranty
AI:

> Finished chain.


"Hi, I'm Julian. What is the problem?"

In [8]:
conv_chain.predict(input="The screen is broken")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, I'm Julian. My TV is broken but is under warranty
AI: Hi, I'm Julian. What is the problem?
Human: The screen is broken
AI:

> Finished chain.


'Is it still under warranty?'

In [9]:
conv_chain.predict(input="Yes, the warranty numbers is A1241")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, I'm Julian. My TV is broken but is under warranty
AI: Hi, I'm Julian. What is the problem?
Human: The screen is broken
AI: Is it still under warranty?
Human: Yes, the warranty numbers is A1241
AI:

> Finished chain.


'Is it a Samsung?'

In [11]:
# Useful commands
from pprint import pprint
pprint(conv_chain.memory.buffer)

("Human: Hi, I'm Julian. My TV is broken but is under warranty\n"
 "AI: Hi, I'm Julian. What is the problem?\n"
 'Human: The screen is broken\n'
 'AI: Is it still under warranty?\n'
 'Human: Yes, the warranty numbers is A1241\n'
 'AI: Is it a Samsung?')


In [3]:
# Tokens
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("google/flan-ul2")

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [14]:
test_input = "Good morning everyone"
# tokenizer([test_input])
tokenizer.tokenize(test_input) # to see how the tokenizer works

['▁Good', '▁morning', '▁everyone']

In [15]:
memory.buffer

"Human: Hi, I'm Julian. My TV is broken but is under warranty\nAI: Hi, I'm Julian. What is the problem?\nHuman: The screen is broken\nAI: Is it still under warranty?\nHuman: Yes, the warranty numbers is A1241\nAI: Is it a Samsung?"

In [16]:
formatted_prompt = conv_chain.prompt.format(input='next input', history=memory.buffer)
formatted_prompt

"The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.\n\nCurrent conversation:\nHuman: Hi, I'm Julian. My TV is broken but is under warranty\nAI: Hi, I'm Julian. What is the problem?\nHuman: The screen is broken\nAI: Is it still under warranty?\nHuman: Yes, the warranty numbers is A1241\nAI: Is it a Samsung?\nHuman: next input\nAI:"

In [9]:
def chat_to_llm(chat_llm):
    conversation_total_tokens = 0
    new_conversation = ConversationChain(llm=chat_llm, 
                                     verbose=False, 
                                     memory=ConversationBufferMemory()
                                     )
    
    while True:
        message = input("Human: ")
        if message=='exit':
            print(f"{conversation_total_tokens} tokens used in total in this conversation")
            break
        if message:
            formatted_prompt = new_conversation.prompt.format(input=message,history=new_conversation.memory.buffer)
            num_tokens = len(tokenizer.tokenize(formatted_prompt))
            conversation_total_tokens += num_tokens
            print(f'tokens sent {num_tokens}')
            response = new_conversation.predict(input=message)
            response_num_tokens = len(tokenizer.tokenize(response))
            conversation_total_tokens += response_num_tokens
            print(f"LLM: {response}")

In [10]:
chat_to_llm(conv_model)

tokens sent 61
LLM: Hi, how can I help you?
tokens sent 76
LLM: I can help you with anything.
tokens sent 94
LLM: I'm sorry to hear that.
tokens sent 110
LLM: Is there anything I can do?
372 tokens used in total in this conversation
